In [71]:
import pandas as pd
import numpy as np

In [72]:
oshima = pd.read_csv('/Users/genie/Documents/COLLABORATION/AirbnbWise/Modeling/Data/Oshima/oshima_data_add_latlng.csv')
display(oshima.head())

,Unnamed: 0,district,address,info,date,full name,latitude,longitude
0,0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309
1,1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166
2,2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623
3,3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401
4,4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044


In [73]:
oshima.drop(columns=['Unnamed: 0'], inplace=True)

In [74]:
#pd.options.display.max_rows = 9999
oshima['info'].value_counts().head(10)

火災による死亡            317
死体発見               238
飛び降り自殺             168
自殺                 142
孤高死                101
告知事項あり（不動産情報より）     94
首吊り自殺               77
転落死                 71
刺殺                  64
告知事項あり              60
Name: info, dtype: int64

In [75]:

#TODO : oshima 범죄 카테고리 상위 10개 
#TODO : < 1차적 목표 > 화재, 시체발견, 자살, 투신자살, 추락사, 부동산 고지사항 카테고리 범주 안에서 태깅 목표(완료)
#TODO : < 2차적 목표 > 상위 10개 제외한 예외 경우 새로운 카테고리 만들어서 태깅
#TODO : < 3차적 목표 > 고지사항의 경우 특정 죽음 키워드(사망, 자살)이 포함된 안내사항 인 경우 태깅하고 그렇지 않으면 데이터 삭제

#? : 告知事項あり（不動産情報より）와 告知事項あり는 같은 부동산 고지사항인데 분류된 기준이 뭐일까? -> 부동산 고지사항에 해당하는 부분 좀 더 파보자.
#* 火災による死亡           317 -> 火災 : 화재
#* 死体発見               238 -> 시체 발견 -> 死体 : 시체 
#* 飛び降り自殺             168 -> 높은 곳에서 뛰어 내려 자살(투신자살) -> 自殺 : 자살, 추가 info
#* 自殺                 142 -> 자살 
#* 孤高死                101 -> 고독사
#* 告知事項あり（不動産情報より）     94 -> 부동산 정보로 나온 정보를 통해 거래 상대방에게 고지한 내용, 좀 더 강조된 걸까?
#* 首吊り自殺               77 -> 목 매달아 자살 -> 자살, 추가 info
#* 転落死                 71 -> 낙하사망(추락사)
#* 刺殺                  64 -> 찔러서 죽인 행위 -> 타살
#* 告知事項あり            60 -> -> 부동산 정보로 나온 정보를 통해 거래 상대방에게 고지한 내용 
#! ['火災', '死体', '自殺', '孤高死', '告知事項あり', '転落死', '刺殺'] -> 화재, 시체, 자살, 고독사, 고지사항, 추락사, 타살

In [76]:
oshima.shape

(4833, 7)

In [77]:
#* 주소 중복 데이터 확인됨
oshima.drop_duplicates(keep='first', subset=['address'], inplace=True)
oshima.shape

(3644, 7)

In [78]:
oshima['info'].value_counts().head(10)

火災による死亡            254
死体発見               178
飛び降り自殺             121
自殺                 106
孤高死                 83
告知事項あり（不動産情報より）     76
首吊り自殺               56
転落死                 51
刺殺                  47
告知事項あり              41
Name: info, dtype: int64

In [79]:
# categoryList = oshima['info'].value_counts().head(10).index.tolist()
# categoryList

In [80]:
#* 상위 10개에 해당하는 데이터와 아닌 데이터로 구분
# oshimaTop10 = oshima[oshima['info'].isin(categoryList)].reset_index(drop=True)
# oshimaNotTop10 = oshima[~oshima['info'].isin(categoryList)].reset_index(drop=True)
# print(oshimaTop10.shape, oshimaNotTop10.shape)

In [81]:
# oshimaTop10.loc[:, 'crimeType'] = np.nan
# oshimaNotTop10.loc[:, 'crimeType'] = np.nan

In [82]:
#* 카테고리 7개로 축소
#! ['火災', '死体', '自殺', '孤高死', '告知事項あり', '転落死', '刺殺'] -> 화재, 시체, 자살, 고독사, 고지사항, 추락사, 타살
# keyword_mapping = {
#     '火災': '화재',
#     '死体': '시체',
#     '自殺': '자살',
#     '孤高死': '고독사',
#     '告知事項あり': '부동산고지',
#     '転落死': '추락사',
#     '刺殺': '타살'
# }

In [83]:
def taggingF(df):
    keyword_mapping = {
        '火災による死亡' : '화재',
        '火災': '화재',
        '死体': '시체',
        '遺体' : '시체',
        'ミイラ' : '시체',
        '自殺': '자살',
        '首吊り' : '자살',
        '飛び降り' : '자살',
        '無理心中' : '자살',
        '縊死' : '자살',
        '孤高死': '고독사',
        '孤独死' : '고독사',
        'て独' : '고독사',
        '告知事項あり': '부동산고지',
        '心理的瑕疵' : '부동산고지',
        '告知事項有り' : '부동산고지',
        '告知事項有' : '부동산고지',
        '告知事項が御座います' : '부동산고지',
        '告知事項' : '부동산고지',
        '転落死': '추락사',
        'から転落' : '추락사',
        '刺殺': '타살',
        '絞殺' : '타살',
        '射殺' : '타살',
        '車両事故死' : '타살',
        '傷害致死' : '타살',
        '撲殺' : '타살',
        '殺人' : '타살',
        '殴殺' : '타살',
        '虐待' : '타살',
        '病死' : '질병사',
        '過労死' : '질병사',
        '熱中症による死亡' : '질병사',
        '事故死' : '사고사',
        'エレベータと壁に挟まれ死亡' : '사고사',
        '変死' : '의문사',
        '不審死' : '의문사', 
        '原因不明の事故' : '의문사',
        '精神的瑕疵' : '심신미약'
        
    }
    df.loc[:, 'crimeType'] = np.nan

    for idx in range(oshima.shape[0]):
        for key, value in keyword_mapping.items():
            try:
                if key in str(df.loc[idx, 'info']):
                    df.loc[idx, 'crimeType'] = value
                    break
            except KeyError:
                    continue
    return df

In [84]:
df2 = taggingF(oshima)
df2

,district,address,info,date,full name,latitude,longitude,crimeType
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,시체
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,자살
...,...,...,...,...,...,...,...,...
4816,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,NaN
4817,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,NaN
4818,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,NaN
4819,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\r\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,NaN


In [86]:
df2.loc[(df2['info'] == '火災による死亡')] #! mapping이 왜 안될까?

,district,address,info,date,full name,latitude,longitude,crimeType
45,新宿区,東京都新宿区歌舞伎町二丁目28-11ホテルまつき1階-2階間踊り場,火災による死亡,平成28年1月7日,東京都 新宿区 歌舞伎町 二丁目 28番 11号,35.696590,139.703430,화재
74,豊島区,東京都豊島区東池袋四丁目18-15フラワーハイツ2階,火災による死亡,平成20年12月20日,東京都 豊島区 東池袋 四丁目 18番 15号,35.726536,139.720749,화재
122,杉並区,東京都杉並区阿佐谷南一丁目20-22川染荘101,火災による死亡,平成24年3月22日,東京都 杉並区 阿佐谷 南一丁目 20番 22号,35.700211,139.640106,화재
132,板橋区,東京都荒川区荒川三丁目30-4,火災による死亡,平成21年12月24日,東京都 荒川区 荒川 三丁目 30番 4号,35.734417,139.781876,화재
150,足立区,東京都千代田区神田神保町一丁目10柏水堂ビル4階,火災による死亡,平成21年7月3日,東京都 千代田区 神田神保町 一丁目 10番地,35.696121,139.758408,화재
...,...,...,...,...,...,...,...,...
4755,調布市,東京都調布市多摩川三丁目36-11パーク多摩川106,火災による死亡,平成26年1月6日,東京都 調布市 多摩川 三丁目 36番,35.646194,139.530014,NaN
4758,杉並区,東京都杉並区下井草二丁目18-19,火災による死亡,平成24年3月11日,東京都 杉並区 下井草 二丁目 18番 19号,35.720215,139.629898,NaN
4767,杉並区,東京都中野区大和町一丁目2-3市瀬荘202,火災による死亡,平成27年5月17日,東京都 中野区 大和町 一丁目 2番 3号,35.709927,139.653458,NaN
4774,杉並区,東京都中野区中野三丁目34-8藤和シティコープ中野501,火災による死亡,平成20年12月1日,東京都 中野区 中野 三丁目 34番 8号,35.703484,139.664795,NaN
